In [43]:
!pwd

/Users/shihosato/src/github.com/twinkle13531/master_degree/202012/Summary


In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import sys, os
sys.path.append(os.path.join('..', 'Random'))
sys.path.append(os.path.join('..', 'SA'))
sys.path.append(os.path.join('..', 'QA'))
sys.path.append(os.path.join('..', '4ti2', 'functions'))

In [46]:
import aws_QA_DwaveSampler_AutoEmbedding as aqa ##new
import random_exact_test_functions as ret
import SA_exact_test_functions as net
import zsolve_t1 as zt
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import statistics
import math

In [47]:
time_dic = {}

In [48]:
y_dic = {'total_y_num':{}, 'mean_occurrence':{}, 'valid_y_num':{}}

In [49]:
per_time_dic = {}

In [50]:
mean_broken_chain_proportion = {}#only QA

In [51]:
p_dic = {}

In [52]:
chain_strength_dic = {20:15, 25:15, 30:15, 35:20, 40:20}
#chain_strength_dic = {20:10, 25:15, 30:20, 35:20, 40:30}

In [53]:
bit_list = [20, 25, 30, 35, 40]
num_list = [0, 1, 2, 3, 4]

## QA
計算時間の制約があるので一気に、まとめて計算する。


前回からの変更点
- calculation_time測定箇所がおかしかった→修正
- n, num_reads.たしかにnを増やすとvalid_y_numが増えるが、per_timeもcalculation_timeも*n弱ほど増える
  - どう頑張ってもvalid_y_numでSAには勝てない
  - n, num_reads = 100, 100となるとどうしても計算が時間かかりすぎる25bitでさえも
  - →n=1, num_reads=10000のtotal_timeで勝負する
  - 時間の平均、標準偏差などはtotal_timeにする（すべての手法）


In [54]:
########################################## 予備実験 ################################################################
#n=1でのみ通用するコード
n, num_reads = 1, 10000
chain_strength_dic = {20:15, 25:15, 30:15, 35:20, 40:20}
time_dic['QA'] = {}
y_dic['total_y_num']['QA'] = {}
y_dic['mean_occurrence']['QA'] = {}
y_dic['valid_y_num']['QA'] = {}
per_time_dic['QA'] = {}
p_dic['QA'] = {}

for bit in [20, 30]:
    time_dic['QA'][bit] = {}
    y_dic['total_y_num']['QA'][bit] = {}
    y_dic['mean_occurrence']['QA'][bit] = {}
    y_dic['valid_y_num']['QA'][bit] = {}
    per_time_dic['QA'][bit] = {}
    mean_broken_chain_proportion[bit] = {}
    p_dic['QA'][bit] = {}
    chain_strength = chain_strength_dic[bit]
    
    for num in [2, 3]:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = aqa.make_Hamiltonian(df)
        res = aqa.aws_get_response(bqm, chain_strength, num_reads)
        ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
        valid_y_info_dic = ads.find_valid_y()
        
        if len(valid_y_info_dic) > 0:
            calculation_time = res.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']*10**(-6)
            occurrence_list = [valid_y_info[0] for valid_y_info in list(valid_y_info_dic.values())]
            broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]

            time_dic['QA'][bit][num] = calculation_time
            y_dic['total_y_num']['QA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['QA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['QA'][bit][num] = len(valid_y_info_dic)
            #per_time_dic['QA'][bit][num] = calculation_time / sum(occurrence_list)
            per_time_dic['QA'][bit][num] = calculation_time / len(valid_y_info_dic)
            mean_broken_chain_proportion[bit][num] = statistics.mean(broken_list)
            p_dic['QA'][bit][num] = ads.p_value(valid_y_info_dic)

    print('{} finish'.format(bit))

20 finish
30 finish


In [55]:
time_dic['QA']

{20: {2: 2.400298, 3: 2.4002719999999997}, 30: {2: 2.400395, 3: 2.400392}}

In [56]:
y_dic['total_y_num']['QA']

{20: {2: 254, 3: 147}, 30: {2: 17, 3: 6}}

In [57]:
y_dic['mean_occurrence']['QA']

{20: {2: 1, 3: 1}, 30: {2: 1, 3: 1}}

In [58]:
y_dic['valid_y_num']['QA']

{20: {2: 185, 3: 140}, 30: {2: 17, 3: 6}}

In [59]:
per_time_dic['QA']

{20: {2: 0.012974583783783782, 3: 0.017144799999999998},
 30: {2: 0.14119970588235295, 3: 0.40006533333333333}}

In [60]:
p_dic['QA']

{20: {2: 0.33513513513513515, 3: 0.18571428571428572}, 30: {2: 0.0, 3: 0.0}}

In [61]:
mean_broken_chain_proportion

{20: {2: 0.0002702702702702703, 3: 0.0007142857142857143},
 30: {2: 0.011764705882352941, 3: 0.005555555555555556}}

In [14]:
############################################################ 本番 ##########################################################################
#n=1でのみ通用するコード
n, num_reads = 1, 10000
chain_strength_dic = {20:15, 25:15, 30:15, 35:20, 40:20}
time_dic['QA'] = {}
y_dic['total_y_num']['QA'] = {}
y_dic['mean_occurrence']['QA'] = {}
y_dic['valid_y_num']['QA'] = {}
per_time_dic['QA'] = {}
p_dic['QA'] = {}

for bit in bit_list:
    time_dic['QA'][bit] = {}
    y_dic['total_y_num']['QA'][bit] = {}
    y_dic['mean_occurrence']['QA'][bit] = {}
    y_dic['valid_y_num']['QA'][bit] = {}
    per_time_dic['QA'][bit] = {}
    mean_broken_chain_proportion[bit] = {}
    p_dic['QA'][bit] = {}
    chain_strength = chain_strength_dic[bit]
    
    for num in num_list:
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        bqm = aqa.make_Hamiltonian(df)
        res = aqa.aws_get_response(bqm, chain_strength, num_reads)
        ads = aqa.aws_DSampler(df, bqm, num_reads, chain_strength, res)
        valid_y_info_dic = ads.find_valid_y()
        
        if len(valid_y_info_dic) > 0:
            calculation_time = res.info['additionalMetadata']['dwaveMetadata']['timing']['qpuAccessTime']*10**(-6)
            occurrence_list = [valid_y_info[0] for valid_y_info in list(valid_y_info_dic.values())]
            broken_list = [valid_y_info[1] for valid_y_info in list(valid_y_info_dic.values())]

            time_dic['QA'][bit][num] = calculation_time
            y_dic['total_y_num']['QA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['QA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['QA'][bit][num] = len(valid_y_info_dic)
            #per_time_dic['QA'][bit][num] = calculation_time / sum(occurrence_list)
            per_time_dic['QA'][bit][num] = calculation_time / len(valid_y_info_dic)
            mean_broken_chain_proportion[bit][num] = statistics.mean(broken_list)
            p_dic['QA'][bit][num] = ads.p_value(valid_y_info_dic)

    print('{} finish'.format(bit))

SolverFailureError: Problem not accepted because user kitai_koki_16@stu-cbms.k.u-tokyo.ac.jp has insufficient remaining solver access time in project TOKY

In [ ]:
time_dic['QA']

In [ ]:
y_dic['total_y_num']['QA']

In [ ]:
y_dic['mean_occurrence']['QA']

In [ ]:
y_dic['valid_y_num']['QA']

In [ ]:
per_time_dic['QA']

In [ ]:
p_dic['QA']

In [ ]:
mean_broken_chain_proportion

## Random

前回からの変更点
- calculation_timeをret.find_valid_y(df, num_reads)の時間とした
```python
time_0 = time.time()
valid_y_dic = ret.find_valid_y(df, num_reads)
time_1 = time.time()
calculation_time = time_1 - time_0
```

In [17]:
num_reads = 10000

time_dic['random'] = {}
y_dic['total_y_num']['random'] = {}
y_dic['mean_occurrence']['random'] = {}
y_dic['valid_y_num']['random'] = {}
per_time_dic['random'] = {}
p_dic['random'] = {}

for bit in bit_list:
    time_dic['random'][bit] = {}
    y_dic['total_y_num']['random'][bit] = {}
    y_dic['mean_occurrence']['random'][bit] = {}
    y_dic['valid_y_num']['random'][bit] = {}
    per_time_dic['random'][bit] = {}
    p_dic['random'][bit] = {}
    
    for num in num_list:
        df_path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(df_path, sep=',')
        t_list = ret.calc_marginals(df)
        
        time_0 = time.time()
        valid_y_dic = ret.find_valid_y(df, num_reads)
        time_1 = time.time()
        calculation_time = time_1 - time_0
        
        if len(valid_y_dic) > 0:
            rm = ret.Random_method(df, valid_y_dic, num_reads, t_list)
            
            mean_occurrence_list = [i for i in list(valid_y_dic.values())]
            
            time_dic['random'][bit][num] = calculation_time
            y_dic['total_y_num']['random'][bit][num] = sum(mean_occurrence_list)
            y_dic['mean_occurrence']['random'][bit][num] = statistics.mean(mean_occurrence_list)
            y_dic['valid_y_num']['random'][bit][num] = len(valid_y_dic)
            per_time_dic['random'][bit][num] = calculation_time / len(valid_y_dic)
            p_dic['random'][bit][num] = rm.p_value()
        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish
35_0 finish
35_1 finish
35_2 finish
35_3 finish
35_4 finish
40_0 finish
40_1 finish
40_2 finish
40_3 finish
40_4 finish


In [18]:
time_dic['random']

{20: {0: 0.18248987197875977,
  1: 0.13700580596923828,
  2: 0.04710507392883301,
  3: 0.16414785385131836,
  4: 0.1400160789489746},
 25: {0: 0.06069207191467285,
  1: 0.08803606033325195,
  2: 0.12667107582092285,
  3: 0.14335298538208008,
  4: 0.06554007530212402},
 30: {0: 0.05747389793395996,
  1: 0.06588578224182129,
  2: 0.12096881866455078,
  4: 0.059448957443237305},
 35: {1: 0.08326125144958496, 2: 0.12446022033691406, 4: 0.08044815063476562},
 40: {0: 0.08915591239929199, 2: 0.0691230297088623}}

In [19]:
y_dic['total_y_num']['random']

{20: {0: 167, 1: 21, 2: 18, 3: 23, 4: 2},
 25: {0: 3, 1: 17, 2: 6, 3: 1, 4: 12},
 30: {0: 1, 1: 1, 2: 9, 4: 7},
 35: {1: 3, 2: 3, 4: 2},
 40: {0: 1, 2: 1}}

In [20]:
y_dic['mean_occurrence']['random']

{20: {0: 1.0060240963855422, 1: 1, 2: 1, 3: 1, 4: 1},
 25: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 30: {0: 1, 1: 1, 2: 1, 4: 1},
 35: {1: 1, 2: 1, 4: 1},
 40: {0: 1, 2: 1}}

In [21]:
y_dic['valid_y_num']['random']

{20: {0: 166, 1: 21, 2: 18, 3: 23, 4: 2},
 25: {0: 3, 1: 17, 2: 6, 3: 1, 4: 12},
 30: {0: 1, 1: 1, 2: 9, 4: 7},
 35: {1: 3, 2: 3, 4: 2},
 40: {0: 1, 2: 1}}

In [22]:
per_time_dic['random']

{20: {0: 0.0010993365781852998,
  1: 0.006524085998535156,
  2: 0.0026169485516018337,
  3: 0.007136863210926886,
  4: 0.0700080394744873},
 25: {0: 0.020230690638224285,
  1: 0.005178591784308939,
  2: 0.02111184597015381,
  3: 0.14335298538208008,
  4: 0.005461672941843669},
 30: {0: 0.05747389793395996,
  1: 0.06588578224182129,
  2: 0.013440979851616753,
  4: 0.008492708206176758},
 35: {1: 0.027753750483194988, 2: 0.04148674011230469, 4: 0.04022407531738281},
 40: {0: 0.08915591239929199, 2: 0.0691230297088623}}

In [23]:
p_dic['random']

{20: {0: 0.20481927710843373,
  1: 0.38095238095238093,
  2: 0.2777777777777778,
  3: 0.2608695652173913,
  4: 0.0},
 25: {0: 0.0,
  1: 0.058823529411764705,
  2: 0.16666666666666666,
  3: 0.0,
  4: 0.08333333333333333},
 30: {0: 0.0, 1: 1.0, 2: 0.1111111111111111, 4: 0.0},
 35: {1: 0.0, 2: 0.0, 4: 0.0},
 40: {0: 0.0, 2: 0.0}}

## SA
- occurrence_list = list(valid_y_info_dic.values())箇所を変更

In [24]:
num_reads = 10000

In [25]:
time_dic['SA'] = {}
y_dic['total_y_num']['SA'] = {}
y_dic['mean_occurrence']['SA'] = {}
y_dic['valid_y_num']['SA'] = {}
per_time_dic['SA'] = {}
p_dic['SA'] = {}

In [26]:
for bit in bit_list:
    time_dic['SA'][bit] = {}
    y_dic['total_y_num']['SA'][bit] = {}
    y_dic['mean_occurrence']['SA'][bit] = {}
    y_dic['valid_y_num']['SA'][bit] = {}
    per_time_dic['SA'][bit] = {}
    p_dic['SA'][bit] = {}
    
    for num in num_list:
        
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        
        res, elapsed_time = net.make_res_data(df, num_reads)
        valid_y_info_dic = net.find_valid_y(res)

        occurrence_list = list(valid_y_info_dic.values())
        
        if len(valid_y_info_dic) > 0:
            rvd = net.SA_res_valid_dic(df, res, valid_y_info_dic, num_reads)
            
            time_dic['SA'][bit][num] = elapsed_time
            y_dic['total_y_num']['SA'][bit][num] = sum(occurrence_list)
            y_dic['mean_occurrence']['SA'][bit][num] = statistics.mean(occurrence_list)
            y_dic['valid_y_num']['SA'][bit][num] = len(valid_y_info_dic)
            per_time_dic['SA'][bit][num] = elapsed_time / len(valid_y_info_dic)
            p_dic['SA'][bit][num] = rvd.calc_p_value()
            
        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish
35_0 finish
35_1 finish
35_2 finish
35_3 finish
35_4 finish
40_0 finish
40_1 finish
40_2 finish
40_3 finish
40_4 finish


In [27]:
time_dic['SA']

{20: {0: 3.898098945617676,
  1: 3.915046215057373,
  2: 3.8129260540008545,
  3: 3.724759817123413,
  4: 4.148306846618652},
 25: {0: 5.367434978485107,
  1: 5.3866660594940186,
  2: 5.084359169006348,
  3: 5.34026575088501,
  4: 5.0264599323272705},
 30: {0: 6.630839109420776,
  1: 6.8572258949279785,
  2: 7.260678291320801,
  3: 6.876424074172974,
  4: 6.8197081089019775},
 35: {0: 8.292114973068237,
  1: 8.319061040878296,
  2: 8.449536085128784,
  3: 8.385251760482788,
  4: 8.262472867965698},
 40: {0: 10.168527126312256,
  1: 10.351256132125854,
  2: 11.018885850906372,
  3: 10.484769105911255,
  4: 10.583085060119629}}

In [28]:
y_dic['total_y_num']['SA']

{20: {0: 9765, 1: 9866, 2: 9855, 3: 9615, 4: 9615},
 25: {0: 9831, 1: 9794, 2: 9402, 3: 8549, 4: 9842},
 30: {0: 9781, 1: 9254, 2: 9937, 3: 9220, 4: 9918},
 35: {0: 9605, 1: 9759, 2: 9810, 3: 9537, 4: 9484},
 40: {0: 9702, 1: 9766, 2: 9703, 3: 9562, 4: 9561}}

In [29]:
y_dic['mean_occurrence']['SA']

{20: {0: 1, 1: 7, 2: 6, 3: 4, 4: 36},
 25: {0: 1, 1: 1, 2: 1, 3: 6, 4: 1},
 30: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 35: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1},
 40: {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}}

In [30]:
y_dic['valid_y_num']['SA']

{20: {0: 7408, 1: 1363, 2: 1634, 3: 2360, 4: 260},
 25: {0: 6795, 1: 9297, 2: 6995, 3: 1368, 4: 7849},
 30: {0: 8968, 1: 8503, 2: 9812, 3: 8872, 4: 9872},
 35: {0: 9552, 1: 9756, 2: 9806, 3: 9521, 4: 9478},
 40: {0: 9701, 1: 9763, 2: 9703, 3: 9561, 4: 9561}}

In [31]:
per_time_dic['SA']

{20: {0: 0.0005262012615574617,
  1: 0.0028723743323971923,
  2: 0.002333492077111906,
  3: 0.0015782880581031412,
  4: 0.015955026333148662},
 25: {0: 0.0007899094891074478,
  1: 0.0005793983069263223,
  2: 0.0007268562071488703,
  3: 0.003903703034272668,
  4: 0.0006403949461494802},
 30: {0: 0.0007393888391414782,
  1: 0.0008064478295810865,
  2: 0.0007399794426539748,
  3: 0.0007750703419942486,
  4: 0.0006908132201075747},
 35: {0: 0.0008681024888053012,
  1: 0.0008527122838128634,
  2: 0.0008616700066417279,
  3: 0.0008807112446678698,
  4: 0.0008717527820179045},
 40: {0: 0.0010481937043925632,
  1: 0.0010602536241038467,
  2: 0.0011356163919309875,
  3: 0.0010966184610303581,
  4: 0.0011069014810291423}}

In [32]:
p_dic['SA']

{20: {0: 0.23663606911447085,
  1: 0.21496698459280997,
  2: 0.3329253365973072,
  3: 0.2639830508474576,
  4: 0.4153846153846154},
 25: {0: 0.08373804267844003,
  1: 0.06356889319135205,
  2: 0.1472480343102216,
  3: 0.15716374269005848,
  4: 0.25595617276086124},
 30: {0: 0.04471454058876004,
  1: 0.3020110549217923,
  2: 0.2109661638809621,
  3: 0.10651487826871055,
  4: 0.06432333873581848},
 35: {0: 0.058103015075376886,
  1: 0.05125051250512505,
  2: 0.012747297572914542,
  3: 0.07614746350173301,
  4: 0.14971512977421397},
 40: {0: 0.0380373157406453,
  1: 0.06043224418723753,
  2: 0.0296815417911986,
  3: 0.06432381550047066,
  4: 0.028971864867691665}}

## exhaustive enumeration

In [34]:
enu_time_dic = {}
enu_time_dic[20] = [4.84, 0.02, 0.23, 0.38, 0.65]
enu_time_dic[25] = [5.47, 2894.31, 4.19, 0.23, 15.37]
enu_time_dic[30] = [120017.00, 105.15, 12792.04, 725.79, 140234.05]

In [35]:
time_dic['enumeration'] = {}
y_dic['valid_y_num']['enumeration'] = {}
per_time_dic['enumeration'] = {}
p_dic['enumeration'] = {}

In [36]:
for bit in [20, 25, 30]:
    time_dic['enumeration'][bit] = {}
    y_dic['valid_y_num']['enumeration'][bit] = {}
    per_time_dic['enumeration'][bit] = {}
    p_dic['enumeration'][bit] = {}
    
    for num in [0, 1, 2, 3, 4]:
        
        path = '../input/ost{}_{}.csv'.format(bit, num)
        df = pd.read_csv(path, sep=',')
        
        zinhom_path = '../4ti2/{}bit/{}/{}bit_{}.zinhom'.format(bit, num, bit, num)
        zinhom_num, zinhom_list = zt.make_result_list(zinhom_path)
        
        if zinhom_num > 0:
            zrs = zt.zsolve_result_summerize(df, zinhom_list)
            
            time_dic['enumeration'][bit][num] = enu_time_dic[bit][num]
            y_dic['valid_y_num']['enumeration'][bit][num] = zinhom_num
            per_time_dic['enumeration'][bit][num] = enu_time_dic[bit][num] / zinhom_num
            p_dic['enumeration'][bit][num] = zrs.calucurate_p()

        print('{}_{} finish'.format(bit, num))

20_0 finish
20_1 finish
20_2 finish
20_3 finish
20_4 finish
25_0 finish
25_1 finish
25_2 finish
25_3 finish
25_4 finish
30_0 finish
30_1 finish
30_2 finish
30_3 finish
30_4 finish


In [37]:
time_dic['enumeration']

{20: {0: 4.84, 1: 0.02, 2: 0.23, 3: 0.38, 4: 0.65},
 25: {0: 5.47, 1: 2894.31, 2: 4.19, 3: 0.23, 4: 15.37},
 30: {0: 120017.0, 1: 105.15, 2: 12792.04, 3: 725.79, 4: 140234.05}}

In [38]:
y_dic['valid_y_num']['enumeration']

{20: {0: 16464, 1: 1365, 2: 1638, 3: 2397, 4: 260},
 25: {0: 12600, 1: 89796, 2: 14700, 3: 1372, 4: 21000},
 30: {0: 58296, 1: 57159, 2: 429660, 3: 127440, 4: 839916}}

In [39]:
per_time_dic['enumeration']

{20: {0: 0.00029397473275024293,
  1: 1.4652014652014653e-05,
  2: 0.00014041514041514042,
  3: 0.00015853149770546517,
  4: 0.0025},
 25: {0: 0.0004341269841269841,
  1: 0.03223205933449151,
  2: 0.0002850340136054422,
  3: 0.00016763848396501459,
  4: 0.0007319047619047618},
 30: {0: 2.058751886921916,
  1: 0.0018396053114995016,
  2: 0.029772471256342227,
  3: 0.00569515065913371,
  4: 0.16696199381842944}}

In [40]:
p_dic['enumeration']

{20: {0: 0.22959183673469388,
  1: 0.2153846153846154,
  2: 0.3333333333333333,
  3: 0.2640801001251564,
  4: 0.4153846153846154},
 25: {0: 0.08333333333333333,
  1: 0.0608044901777362,
  2: 0.15306122448979592,
  3: 0.15743440233236153,
  4: 0.248},
 30: {0: 0.04459997255386304,
  1: 0.31648559282002836,
  2: 0.20353302611367127,
  3: 0.11007532956685499,
  4: 0.0675067506750675}}

## mean, std dicについて
 - enumeration入(all_mean_time, all_std_time)と、なしを作る
   - 入はせっかくなので4サンプルずつ20, 25, 30bitまで
   - なしは5サンプルずつ20-40bitまで
 - 種類
   - time
   - per_time
   - valid_y_num
   - p_value//
   
   - total_y_num
   - mean_occurrence

In [ ]:
method_list_all = ['enumeration', 'random', 'SA', 'QA']
method_list_35 = ['random', 'SA', 'QA']
method_list_40 = ['SA', 'QA']

bit_list_30 = [20, 25, 30]
bit_list_35 = [20, 25, 30, 35]
bit_list_40 = [20, 25, 30, 35, 40]
#bit_list_all = [20, 25, 30, 35, 40]

In [ ]:
import statistics
import numpy as np

### time

In [ ]:
mean_time_30 = {}
std_time_30 = {}
for method in method_list_all:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_30:
        time_list = list(time_dic[method][bit].values())[:4]
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_30[method] = np.array(mean_time_list)
    std_time_30[method] = np.array(std_time_list)

In [ ]:
mean_time_30

In [ ]:
std_time_30

In [ ]:
mean_time_35 = {}
std_time_35 = {}
for method in method_list_35:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_35:
        time_list = list(time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_35[method] = np.array(mean_time_list)
    std_time_35[method] = np.array(std_time_list)

In [ ]:
mean_time_35

In [ ]:
std_time_35

In [ ]:
mean_time_40 = {}
std_time_40 = {}
for method in method_list_40:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_40:
        time_list = list(time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_time_40[method] = np.array(mean_time_list)
    std_time_40[method] = np.array(std_time_list)

In [ ]:
mean_time_40

In [ ]:
std_time_40

### per_time

In [ ]:
mean_per_time_30 = {}
std_per_time_30 = {}

for method in method_list_all:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_30:
        time_list = list(per_time_dic[method][bit].values())[:4]
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_30[method] = np.array(mean_time_list)
    std_per_time_30[method] = np.array(std_time_list)

In [ ]:
mean_per_time_30

In [ ]:
std_per_time_30

In [ ]:
mean_per_time_35 = {}
std_per_time_35 = {}

for method in method_list_35:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_35:
        time_list = list(per_time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_35[method] = np.array(mean_time_list)
    std_per_time_35[method] = np.array(std_time_list)

In [ ]:
mean_per_time_35

In [ ]:
std_per_time_35

In [ ]:
mean_per_time_40 = {}
std_per_time_40 = {}

for method in method_list_40:
    mean_time_list = []
    std_time_list = []
    for bit in bit_list_40:
        time_list = list(per_time_dic[method][bit].values())
        mean_time_list.append(statistics.mean(time_list))
        std_time_list.append(statistics.pstdev(time_list))

    mean_per_time_40[method] = np.array(mean_time_list)
    std_per_time_40[method] = np.array(std_time_list)

In [ ]:
mean_per_time_40

In [ ]:
std_per_time_40

### valid_y_num

In [ ]:
mean_valid_y_num_30 = {}
std_valid_y_num_30 = {}

for method in method_list_all:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_30:
        num_list = list(y_dic['valid_y_num'][method][bit].values())[:4]
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_30[method] = np.array(mean_num_list)
    std_valid_y_num_30[method] = np.array(std_num_list)

In [ ]:
mean_valid_y_num_30

In [ ]:
std_valid_y_num_30

In [ ]:
mean_valid_y_num_35 = {}
std_valid_y_num_35 = {}

for method in method_list_35:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_35:
        num_list = list(y_dic['valid_y_num'][method][bit].values())
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_35[method] = np.array(mean_num_list)
    std_valid_y_num_35[method] = np.array(std_num_list)

In [ ]:
mean_valid_y_num_35

In [ ]:
std_valid_y_num_35

In [ ]:
mean_valid_y_num_40 = {}
std_valid_y_num_40 = {}

for method in method_list_40:
    mean_num_list = []
    std_num_list = []
    for bit in bit_list_40:
        num_list = list(y_dic['valid_y_num'][method][bit].values())
        mean_num_list.append(statistics.mean(num_list))
        std_num_list.append(statistics.pstdev(num_list))

    mean_valid_y_num_40[method] = np.array(mean_num_list)
    std_valid_y_num_40[method] = np.array(std_num_list)

In [ ]:
mean_valid_y_num_40

In [ ]:
std_valid_y_num_40

### p_value

In [ ]:
mean_p_30 = {}
std_p_30 = {}

for method in method_list_all:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_30:
        p_list = list(p_dic[method][bit].values())[:4]
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_30[method] = np.array(mean_p_list)
    std_p_30[method] = np.array(std_p_list)

In [ ]:
mean_p_30

In [ ]:
std_p_30

In [ ]:
mean_p_35 = {}
std_p_35 = {}

for method in method_list_35:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_35:
        p_list = list(p_dic[method][bit].values())
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_35[method] = np.array(mean_p_list)
    std_p_35[method] = np.array(std_p_list)

In [ ]:
mean_p_35

In [ ]:
std_p_35

In [ ]:
mean_p_40 = {}
std_p_40 = {}

for method in method_list_40:
    mean_p_list = []
    std_p_list = []
    for bit in bit_list_40:
        #print(bit)
        p_list = list(p_dic[method][bit].values())
        mean_p_list.append(statistics.mean(p_list))
        std_p_list.append(statistics.pstdev(p_list))

    mean_p_40[method] = np.array(mean_p_list)
    std_p_40[method] = np.array(std_p_list)

In [ ]:
mean_p_40

In [ ]:
std_p_40

### total_y_num

In [ ]:
mean_total_y_num_35 = {}
std_total_y_num_35 = {}

for method in method_list_35:
    mean_total_num_list = []
    std_total_num_list = []
    for bit in bit_list_35:
        total_num_list = list(y_dic['total_y_num'][method][bit].values())
        mean_total_num_list.append(statistics.mean(total_num_list))
        std_total_num_list.append(statistics.pstdev(total_num_list))

    mean_total_y_num_35[method] = np.array(mean_total_num_list)
    std_total_y_num_35[method] = np.array(std_total_num_list)

In [ ]:
mean_total_y_num_35

In [ ]:
std_total_y_num_35

In [ ]:
mean_total_y_num_40 = {}
std_total_y_num_40 = {}

for method in method_list_40:
    mean_total_num_list = []
    std_total_num_list = []
    for bit in bit_list_40:
        total_num_list = list(y_dic['total_y_num'][method][bit].values())
        mean_total_num_list.append(statistics.mean(total_num_list))
        std_total_num_list.append(statistics.pstdev(total_num_list))

    mean_total_y_num_40[method] = np.array(mean_total_num_list)
    std_total_y_num_40[method] = np.array(std_total_num_list)

In [ ]:
mean_total_y_num_40

In [ ]:
std_total_y_num_40

### mean_occurrence

In [ ]:
mean_occurrence_35 = {}
std_occurrence_35 = {}

for method in method_list_35:
    mean_occurrence_list = []
    std_occurrence_list = []
    for bit in bit_list_35:
        occurrence_list = list(y_dic['mean_occurrence'][method][bit].values())
        mean_occurrence_list.append(statistics.mean(occurrence_list))
        std_occurrence_list.append(statistics.pstdev(occurrence_list))

    mean_occurrence_35[method] = np.array(mean_occurrence_list)
    std_occurrence_35[method] = np.array(std_occurrence_list)

In [ ]:
mean_occurrence_35

In [ ]:
std_occurrence_35

In [ ]:
mean_occurrence_40 = {}
std_occurrence_40 = {}

for method in method_list_40:
    mean_occurrence_list = []
    std_occurrence_list = []
    for bit in bit_list_40:
        occurrence_list = list(y_dic['mean_occurrence'][method][bit].values())
        mean_occurrence_list.append(statistics.mean(occurrence_list))
        std_occurrence_list.append(statistics.pstdev(occurrence_list))

    mean_occurrence_40[method] = np.array(mean_occurrence_list)
    std_occurrence_40[method] = np.array(std_occurrence_list)

In [ ]:
mean_occurrence_40

In [ ]:
std_occurrence_40